# Week 10: Evaluating Random Number Generators

<font size="6"> Laboratory 7 </font> <br>
<font size="3"> Last updated May 8, 2023 </font>

## <span style="color:orange;"> 00. Content </span>

<font size="5"> Mathematics </font>
- N/A
     
<font size="5"> Programming Skills </font>
- Functions
- Arrays
    
<font size="5"> Embedded Systems </font>
- Thonny & MicroPython

## <span style="color:orange;"> 0. Required Hardware </span>

- see Lab 6

<h3 style="background-color:lightblue"> Write your name and email below: </h3>

**Name:** 

**Email:** 

## <span style="color:orange;">1. Mathematical Tests for RNGs </span>

Given the importance of random number generators (RNGs) in areas such as cryptography, it is clear that there is a need for secure RNGs. The National Institute of Standards and Technology (NIST) has developed formal tests to assist in evaluating the randomness of a randomly generated binary sequence consisting of 0s and 1s. In this lab, we will implement several of these NIST tests and attempt to determine which of the RNGs from the previous lab perform the best.

In [ ]:
import numpy as np

# importing the first 3 rows of the data as a string
# prints the first 3 values as one string
x = np.loadtxt("rng_test_data.txt", dtype=str, max_rows=3) 
seq = ''.join(x)
print(seq)

# importing all data as a floating point number
# prints the first 3 values of the numpy array
y = np.loadtxt("rng_test_data.txt", dtype=str)
y = np.array([float(int(i, 2)) for i in y])
print(y[:3])



### Monobit Frequency Test

An ideal binary random number generator would have approximately the same number of ones and zeros in a sequence. The Monobit Frequency Test quantifies how close the fraction of ones in the sequence matches $\frac{1}{2}$. The steps of the test are as follows:

1. For each bit within every binary data value $X_1,X_2,\dots,X_{n}$, where $X_i$ is a binary bit, apply the transformation $Y_i = 2X_i - 1$ for $i=1,2,\dots,n$.
1. Compute the values of $s$ using the formula: $$ s = \frac{|Y_1+Y_2+\cdots+Y_n|}{\sqrt{n}}.$$
1. If $s > 2.57583$, then conclude that the sequence $(X_1,X_2,\dots,X_n)$ is non-random. The best-case scenario for this computation is when $s = 0$, indicating an equal number of 1s and 0s in the sequence.

According to NIST, the parameter $n$ should be at least 100.

### <span style="color:red"> Exercise 1</span>

__Part 1:__ In Step 1, describe how the transformation affects the ones and zeros in the original random sequence.

__Part 2:__ If $|Y_1+Y_2+\cdots+Y_n|$ is large, what does that imply about the original sequence $(X_1,X_2,\dots,X_{n})$?

__Part 3:__ Implement the Monobit Frequency Test on the test data.

<h3 style="background-color:lightblue"> Write Answers for Exercise 1 Below </h3>


### Monobit Block Frequency Test

This test also tests the proportion of ones and zeros in the random sequence but in size $M$ blocks instead of the entire sequence. Here are the steps:

1. Separate the binary data values $X_1,X_2,\dots,X_{M\cdot N}$ into $N$ non-overlapping blocks of length $M$.
1. For each block, compute the proportion of ones that occur in the length $M$ sequence. 
Denote the proportion of ones in block $i$ by $\pi_i$ for $i=1,2,\dots,N$.
1. Compute $$ t = 4M \sum_{i=1}^N \left(\pi_i-\frac12 \right)^2. $$
1. If $G(N/2,t/2) < 0.01$, then conclude that the sequence $(X_1,X_2,\dots,X_{M\cdot N})$ is non-random. The function $G$ is the normalized incomplete Gamma function.

NIST recommends $20 \leq M < 100$ and $N \leq 100$.

### <span style="color:red"> Exercise </span>

__Part 1:__ If $t$ is large, what does that imply about the original sequence $(X_1,X_2,\dots,X_{M\cdot N})$

__Part 2:__ Choose appropriate values for $M$ and $N$. Implement the Monobit Block Frequency Test on the test data.

<h3 style="background-color:lightblue"> Write Answer for Exercise Below </h3>

In [ ]:
# here's how to import and use the normalized incomplete Gamma function.
from scipy.special import gammaincc as G

G(N/2, t/2) # you will first need to define N and t

### Cumulative Sum Test

This test helps determine if the cumulative sum of the adjusted partial sequences is too large or small. For many non-random sequences the cumulative sums will stray from zero. Here are the steps:

1. With binary data values $X_1,X_2,\dots,X_{n}$, apply the transformation $Y_i = 2X_i - 1$ for $i=1,2,\dots,n$.
1. Compute the forward partial sums
\begin{align*}
    S_1 &= Y_1 \\
    S_2 &= Y_1+Y_2 \\
    S_3 &= Y_1+Y_2+Y_3\\
    \vdots \\
    S_n &= Y_1 + Y_2 + \cdots Y_n \\
\end{align*} 
and the backward partial sums
\begin{align*}
    T_1 &= Y_n \\
    T_2 &= Y_n+Y_{n-1} \\
    \vdots \\
    T_n &= Y_n + Y_{n-1} + \cdots Y_1 \\
\end{align*}
1. Find $$z_{\text{forward}} = \max\{|S_1|,|S_2|,\dots,|S_n|\}, \quad z_{\text{backward}} = \max\{|T_1|,|T_2|,\dots,|T_n|\}.$$
1. Compute
\begin{equation*}
    \rho = 1 -
        \sum_{k=\lfloor \frac{-n/z+1}{4} \rfloor}^{\lfloor \frac{n/z-1}{4} \rfloor} 
            \left[ \Phi\left(\frac{(4k+1)z}{\sqrt{n}}\right) - 
                    \Phi\left(\frac{(4k-1)z}{\sqrt{n}}\right) \right]  +
        \sum_{k=\lfloor \frac{-n/z-3}{4} \rfloor}^{\lfloor \frac{n/z-1}{4} \rfloor} 
            \left[ \Phi\left(\frac{(4k+3)z}{\sqrt{n}}\right) - 
                    \Phi\left(\frac{(4k+1)z}{\sqrt{n}}\right) \right]
\end{equation*} 
for $z=z_{\text{forward}}$ and $z=z_{\text{backward}}$.
The function $\Phi$ is the cumulative distribution function (cdf) of the standard normal distribution, and $\lfloor \cdot \rfloor$ is the floor function which returns the largest integer less than or equal to the input.
1. If $\rho < 0.01$, then conclude that the sequence $(X_1,X_2,\dots,X_{n})$ is non-random.


### <span style="color:red"> Exercise </span>

Choose a value for $n$. NIST suggests $n \geq 100$. Implement the Cumulative Sum Test forwards and backwards on the test data.

<h3 style="background-color:lightblue"> Write Answer for Exercise Below </h3>

In [ ]:
# here's how to import and use the cdf of the standard normal distribution
from scipy.stats import norm
import numpy as np

norm.cdf(x) # you will first need to define x 

np.floor(1.2)

### Runs Test

The runs test measures how much the random sequence oscillates between zero and one. The test formalizes that for a true random sequence, the probability the next value in the sequence is different than the previous value follows a binomial distribution.

A run is a series of values of one kind. For example, $0011010000100111$ has 8 runs. 
Here are the steps:

1. Count the number of runs $R$ in the sequence $(X_1,X_2,\dots,X_n)$.
1. Let $n_0$ be the number of zeros and $n_1$ the number of ones in the sequence. Compute $$q = \frac{|R - \overline{R}|}{u}$$ where $$\overline{R} = \frac{2n_0n_1}{n}+1,\quad u^2=\frac{2n_0n_1(2n_0n_1-n_0-n_1)}{n^2(n+1)}.$$
$\overline{R}$ is the expected number of runs in the sequence.
1. If $q > 2.57583$, then conclude that the sequence $(X_1,X_2,\dots,X_n)$ is non-random.

### <span style="color:red"> Exercise </span>

Choose a value for $n$. NIST suggests $n \geq 100$. Implement the Runs test on the test data.

<h3 style="background-color:lightblue"> Write Answer for Exercise Below </h3>

### <span style="color:red"> Exercise </span>

We will say that a random sequence passes if all 5 tests (Monobit Frequency, Monobit Block Frequency, Forward Cumulative Sum, Backward Cumulative Sum, and Runs) pass.

Compare the pass rates among the random number generators using Method A, B, C, and D based on at least 10 runs (that's 40 random sequences in total from data you collected).

In two paragraphs, summarize your results. Include details of the parameters you chose for the tests and RNGs. Which generators seem to be doing well? Is there one test that fails frequently? Do you think further testing is necessary? Why or why not?

<h3 style="background-color:lightblue"> Write Answer for Exercise Below </h3>

In [ ]:
# to help manage your files,
# in the same directory as this lab .ipynb file, create a folder called folder_of_data (or whatever name you like)
# and move rng_test_data.txt into the folder
# then you can load your data with the updated path to rng_test_data.txt
x = np.loadtxt("folder_of_data/rng_test_data.txt", dtype=str, max_rows=3) 
seq = ''.join(x)
print(seq)

# another tip for file naming
# it is easy to load your files if you name them well
for run_number in range(1,10+1):
    print(f"method_A/run_{run_number}.txt")

## <span style="color:green;"> Reflection </span>

1. Are there any concepts or functions in this lab that you didn't know previously that you believe may be very helpful in later labs?
2. Which part of the lab did you find the most challenging?
3. Which part of the lab was the easiest?

<h3 style="background-color:lightblue"> Write Answers for the Reflection Below </h3>